 # Model Demo Showcase
 this notebook is part of the showcase for the mid semester checkpoint.
 

In [2]:
# all imports (collapsed)
import sqlite3
import pickle
import numpy as np
import re
import random
from time import strftime, localtime

from keras.preprocessing.text import text_to_word_sequence
from keras.utils import pad_sequences

from keras.models import Sequential, load_model
from keras.layers import Embedding, Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint

import re
import requests
from bs4 import BeautifulSoup
from keras.models import load_model

2023-03-16 13:04:03.062770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 13:04:05.418740: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/andyd/miniconda3/envs/nlp/lib/
2023-03-16 13:04:05.422198: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/andyd/miniconda3/envs/nlp/lib/
2023-03-16 13:04:05.422246: W tensorflow/compiler/tf2t

In [3]:
MAX_ARTICLE_LENGTH = 10000
PAD_VALUE = 0

db = sqlite3.connect('text_encoder.db')
c = db.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS "TextEncoder"(
    "word" TEXT,
    "n" INTEGER,
    PRIMARY KEY("word")
);''')
del c

def clean_text(raw_text, to_lower=True):
    text = re.sub(r'(https|http):[^\s]*', ' ', raw_text)
    text = re.sub('[' + re.escape('#@!"$%&(\')*+,-./:;<=>?[\\]^_`{|}~') + ']', '', text)
    text = text.encode('utf8').decode('ascii','ignore')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text.lower()

def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, features='lxml')
    p_tags = soup.find_all('p')
    raw_text = ' '.join([p.text for p in p_tags])
    text = clean_text(raw_text)
    return text

def get_max_key():
    c = db.cursor()
    c.execute('SELECT MAX(n) FROM TextEncoder;')
    return c.fetchone()[0]

def add_word_encoding(word, key):
    c = db.cursor()
    c.execute('INSERT INTO TextEncoder (word, n) VALUES (?, ?);', (word, key))
    assert c.rowcount > 0, "ERROR Inserting key %10s as word: %20s" % (key, word)
    db.commit()
    return


def get_word_key(word, freeze):
    c = db.cursor()
    c.execute('SELECT n FROM TextEncoder WHERE word = ?;', (word,))
    response = c.fetchone()
    if response is not None:
        return response[0]
    else:
        if freeze:
            return None
        else:
            key = get_max_key() + 1
            add_word_encoding(word, key)
            print('Adding word "%20s" as n: %10i' % (word, key))
            return key


def numberize_article(article, freeze):
    tokens = text_to_word_sequence(article)
    num_article = []
    for token in tokens:
        key = get_word_key(token, freeze)
        if key is not None:
            num_article.append(key)
    return np.array(num_article)


def parse_articles(articles, max_length=10000, min_length=20, verbose=False, freeze=False):
    numerified_articles = []
    removed_articles = set()
    for i, a in enumerate(articles):
        numberized = numberize_article(a, freeze)
        if len(numberized) >= min_length:
            numerified_articles.append(numberized)
        else:
            removed_articles.add(i)
            if verbose:
                print('Removing article: %6i for being too short. Length: %3i' % (i, len(numberized)))
    numerified_articles = np.array(numerified_articles)
    return pad_sequences(numerified_articles, maxlen=max_length, dtype='float32', value=0), removed_articles


In [4]:
model = load_model('savedModels/article_lstm_model_230316.115129.h5') #output hidden to remove clutter

2023-03-16 13:04:07.818140: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-16 13:04:07.931066: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-16 13:04:07.931130: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-16 13:04:07.933232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10000, 100)        17328600  
                                                                 
 dropout (Dropout)           (None, 10000, 100)        0         
                                                                 
 conv1d (Conv1D)             (None, 10000, 32)         9632      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 5000, 32)         0         
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, 5000, 32)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                        

In [6]:
def make_prediction(text, model=model):
    parsed_article, _ = parse_articles([text], min_length=0, freeze=True)
    if len(parsed_article) < 1:
        print('Sorry! It looks like once I processed that article, it was too short to use. :\'(')
        return
    print('🤔 Let me think…')
    prediction = model.predict(parsed_article)
    prediction_pct = prediction[0] * 100
    print('''\n   ------------------------------------------------------------------------------------------\n  |   I\'d say that there\'s a %5.2f%% chance that your article is from a right-leaning source. |\n  |  ----------------------------------------------------------------------------------------\n  |/\n🖥️\n''' % prediction_pct)
    return

In [7]:
# Analyze articles from a set test script
test_urls = ['https://www.washingtonpost.com/politics/2023/03/15/trump-fox-news-dominion-defamation-lawsuit/',
        'https://www.washingtonpost.com/politics/2023/03/15/biden-medicare-prescription-drugs-fight/']
for test in test_urls: make_prediction(test)

🤔 Let me think…


2023-03-16 13:04:12.763510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


1/1 [==============================] - 4s 4s/step

   ------------------------------------------------------------------------------------------
  |   I'd say that there's a 99.71% chance that your article is from a right-leaning source. |
  |  ----------------------------------------------------------------------------------------
  |/
🖥️

🤔 Let me think…
1/1 [==============================] - 0s 129ms/step

   ------------------------------------------------------------------------------------------
  |   I'd say that there's a 76.57% chance that your article is from a right-leaning source. |
  |  ----------------------------------------------------------------------------------------
  |/
🖥️



In [8]:
# Analyze articles from any url input
url = input('Enter a url: ').strip()
print('\n\n')
make_prediction(url)




🤔 Let me think…
1/1 [==============================] - 0s 157ms/step

   ------------------------------------------------------------------------------------------
  |   I'd say that there's a 60.40% chance that your article is from a right-leaning source. |
  |  ----------------------------------------------------------------------------------------
  |/
🖥️

